In [1]:
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
import numpy as np

def polygon_check(actual_point, polygon_points):
    '''
    Constructs convex hull from list of tuples of points (polygon_points)
    that define a polygon and determines whether 
    actual_point is within the hull.
    Inputs:
        actual_point (tuple): test point.
        polygon_points (list of tuples): Defines polygon;
        assumed (lat, lon) pairs
    Returns:
        mask[0] (boolean): whether point is within polygon.
    '''
    x = [point[0] for point in polygon_points]
    y = [point[1] for point in polygon_points]
    for point in polygon_points:
        x.append(point[0])
        y.append(point[1])
    hull_points = np.column_stack((x,y))
    predict_point = np.array([[actual_point[0],actual_point[1]]])

    if not isinstance(hull_points,Delaunay):
            hull = Delaunay(hull_points)
    mask = hull.find_simplex(predict_point)>=0
    return mask[0]


def polygon_check2(actual_points, polygon_points):
    '''
    Constructs convex hull from list of tuples of points (polygon_points)
    that define a polygon and determines whether 
    list of points (actual_points) are within the hull.
    Inputs:
        actual_points (list of tuples): test points.
        polygon_points (list of tuples): define polygon;
        assumed (lat, lon) pairs
    Returns:
        mask (pandas mask)): whether points are within polygon.
    '''
    polygon_x = [point[0] for point in polygon_points]
    polygon_y = [point[1] for point in polygon_points]
    actual_points_x = [point[0] for point in actual_points]
    actual_points_y = [point[1] for point in actual_points]
    hull_points = np.column_stack((polygon_x,polygon_y))
    predict_point = np.column_stack((actual_points_x,actual_points_y))

    if not isinstance(hull_points,Delaunay):
            hull = Delaunay(hull_points)
    mask = hull.find_simplex(predict_point)>=0
    return mask

In [2]:
_log_file='../../backup/crime_log_small.csv'

In [4]:
import cynet as cn
import pandas as pd
import numpy as np

df = pd.read_csv(_log_file,parse_dates=True)

In [5]:
df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,8316800,HT550945,08/11/2011 11:00:00 AM,086XX S MARQUETTE AVE,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,False,False,...,7,46,10,1195654,1848294,2011,02/04/2016 06:33:39 AM,41.738615,-87.558742,"(41.738615478, -87.558741896)"
1,8316805,HT550781,10/20/2011 05:00:00 AM,056XX S ABERDEEN ST,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,16,68,06,1169943,1867457,2011,02/04/2016 06:33:39 AM,41.791798,-87.652385,"(41.791797599, -87.652385205)"
2,8316806,HT550706,10/20/2011 05:45:00 AM,079XX S LOOMIS BLVD,031A,ROBBERY,ARMED: HANDGUN,STREET,False,False,...,21,71,03,1168370,1852331,2011,02/04/2016 06:33:39 AM,41.750324,-87.658588,"(41.750323974, -87.658588247)"
3,8316811,HT539324,10/12/2011 12:23:52 PM,003XX E 75TH ST,2027,NARCOTICS,POSS: CRACK,SMALL RETAIL STORE,True,False,...,6,69,18,1179641,1855355,2011,02/04/2016 06:33:39 AM,41.758372,-87.617194,"(41.758372192, -87.61719416)"
4,8316814,HT551005,10/20/2011 10:40:00 AM,103XX S ELIZABETH ST,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,21,73,08B,1169846,1836216,2011,02/04/2016 06:33:39 AM,41.706070,-87.653646,"(41.706070186, -87.653645803)"
5,8316821,HT551028,10/20/2011 10:50:00 AM,040XX W VAN BUREN ST,3731,INTERFERENCE WITH PUBLIC OFFICER,OBSTRUCTING IDENTIFICATION,STREET,True,False,...,24,26,24,1149599,1897719,2011,02/04/2016 06:33:39 AM,41.875259,-87.726199,"(41.875258543, -87.726199383)"
6,8316822,HT551031,10/19/2011 02:00:00 AM,071XX W DICKENS AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,SIDEWALK,False,False,...,36,25,07,1127877,1913161,2011,02/04/2016 06:33:39 AM,41.918028,-87.805607,"(41.918027518, -87.805606689)"
7,8316824,HT551032,10/20/2011 12:00:00 AM,034XX N NATCHEZ AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,...,36,17,26,1132429,1922272,2011,02/04/2016 06:33:39 AM,41.942951,-87.788669,"(41.94295109, -87.788669409)"
8,8316825,HT549690,10/19/2011 12:51:00 PM,079XX S ADA ST,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,...,21,71,26,1168711,1852015,2011,02/04/2016 06:33:39 AM,41.749449,-87.657348,"(41.749449482, -87.657347764)"
9,8316826,HT549865,10/19/2011 06:00:00 AM,011XX N LEAMINGTON AVE,0810,THEFT,OVER $500,RESIDENTIAL YARD (FRONT/BACK),False,False,...,37,25,06,1141821,1907155,2011,02/04/2016 06:33:39 AM,41.901300,-87.754524,"(41.9012995, -87.754523767)"


In [87]:
polygon = [(41.65, -87.68),(41.75,-87.68),(41.65,-87.54),(41.75,-87.54)]

In [8]:
from shapely.geometry import Point, Polygon
import geopandas

In [11]:
"loc_pair" not in df.columns

True

In [39]:
# df["loc_pair"] = [Point(x) for x in list((zip(df["Latitude"], df["Longitude"])))]
# points = list((zip(df["Latitude"], df["Longitude"])))
# new_points = [Point(x) for x in points]
data_geopoints = geopandas.GeoDataFrame(geometry=[Point(x) for x in list((zip(df["Longitude"], df["Latitude"])))])

In [46]:
type(data_geopoints)

geopandas.geodataframe.GeoDataFrame

In [16]:
poly = geopandas.GeoSeries({
    'square': Polygon(polygon)
})

In [17]:
poly

square    POLYGON ((41.5 -87.8, 41.53 -87.8, 41.5 -87.83...
dtype: object

In [91]:
# data_geoppoints = data_geopoints.assign(**{key: pnts.within(geom) for key, geom in poly.items()})
# df["poly_filter"] = data_geopoints.within(Polygon(polygon))
True in data_geopoints.within(Polygon(polygon))

True

In [89]:
df.loc[df["poly_filter"]]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,poly_filter
4,8316814,HT551005,10/20/2011 10:40:00 AM,103XX S ELIZABETH ST,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,73,08B,1169846,1836216,2011,02/04/2016 06:33:39 AM,41.706070,-87.653646,"(41.706070186, -87.653645803)",True
34,8316884,HT544972,10/16/2011 04:30:00 AM,103XX S HALSTED ST,1310,CRIMINAL DAMAGE,TO PROPERTY,SMALL RETAIL STORE,False,False,...,73,14,1172807,1836192,2011,02/04/2016 06:33:39 AM,41.705940,-87.642804,"(41.705939683, -87.642803521)",True
51,8316912,HT550919,10/19/2011 09:30:00 PM,096XX S GREEN ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENTIAL YARD (FRONT/BACK),False,False,...,73,14,1172345,1840843,2011,02/04/2016 06:33:39 AM,41.718713,-87.644359,"(41.718712866, -87.64435905)",True
57,8316919,HT551141,10/20/2011 11:50:00 AM,111XX S ASHLAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,75,08B,1167337,1830440,2011,02/04/2016 06:33:39 AM,41.690274,-87.662998,"(41.690273866, -87.662998245)",True
58,8316921,HT551082,10/20/2011 11:20:00 AM,100XX S MAY ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,True,...,73,14,1170428,1838136,2011,02/04/2016 06:33:39 AM,41.711326,-87.651459,"(41.711326335, -87.65145888)",True
61,8316927,HT550899,05/05/2011 09:00:00 AM,010XX W 109TH ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,75,11,1171191,1832528,2011,02/04/2016 06:33:39 AM,41.695921,-87.648828,"(41.695920504, -87.648827836)",True
69,8316954,HT551090,10/20/2011 11:50:00 AM,088XX S MUSKEGON AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,...,46,08B,1196547,1846833,2011,02/04/2016 06:33:39 AM,41.734584,-87.555519,"(41.734584273, -87.55551858)",True
91,8316993,HT549180,10/18/2011 08:00:00 PM,106XX S HOXIE AVE,0810,THEFT,OVER $500,STREET,False,False,...,51,06,1195211,1834998,2011,02/04/2016 06:33:39 AM,41.702141,-87.560802,"(41.702140957, -87.560802048)",True
93,8316998,HT550833,10/18/2011 02:00:00 PM,090XX S BRANDON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,46,14,1198897,1845558,2011,02/04/2016 06:33:39 AM,41.731027,-87.546952,"(41.731026972, -87.546952093)",True


In [62]:
df = pd.read_csv(_log_file,parse_dates=True)

import matplotlib.pyplot as plt
df.plot(kind="scatter", x="Longitude", y="Latitude", alpha=0.4)
plt.show()

In [81]:
df.loc[(df["Latitude"] > 41.65)
        & (df["Latitude"] <= 41.75)
        & (df["Longitude"] > -87.68)
        & (df["Longitude"] <= -87.54)]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,8316800,HT550945,08/11/2011 11:00:00 AM,086XX S MARQUETTE AVE,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,False,False,...,7,46,10,1195654,1848294,2011,02/04/2016 06:33:39 AM,41.738615,-87.558742,"(41.738615478, -87.558741896)"
4,8316814,HT551005,10/20/2011 10:40:00 AM,103XX S ELIZABETH ST,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,21,73,08B,1169846,1836216,2011,02/04/2016 06:33:39 AM,41.706070,-87.653646,"(41.706070186, -87.653645803)"
8,8316825,HT549690,10/19/2011 12:51:00 PM,079XX S ADA ST,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,False,...,21,71,26,1168711,1852015,2011,02/04/2016 06:33:39 AM,41.749449,-87.657348,"(41.749449482, -87.657347764)"
10,8316827,HT550963,09/01/2011 04:00:00 PM,079XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE-GARAGE,False,False,...,21,71,05,1168380,1851969,2011,02/04/2016 06:33:39 AM,41.749330,-87.658562,"(41.749330381, -87.658562005)"
34,8316884,HT544972,10/16/2011 04:30:00 AM,103XX S HALSTED ST,1310,CRIMINAL DAMAGE,TO PROPERTY,SMALL RETAIL STORE,False,False,...,34,73,14,1172807,1836192,2011,02/04/2016 06:33:39 AM,41.705940,-87.642804,"(41.705939683, -87.642803521)"
40,8316893,HT551023,10/20/2011 02:00:00 AM,081XX S STEWART AVE,0810,THEFT,OVER $500,STREET,False,False,...,21,44,06,1175042,1850896,2011,02/04/2016 06:33:39 AM,41.746240,-87.634182,"(41.746239973, -87.634181801)"
44,8316899,HT550695,10/20/2011 05:30:00 AM,122XX S HALSTED ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE PORCH/HALLWAY,False,False,...,34,53,14,1173210,1823636,2011,02/04/2016 06:33:39 AM,41.671475,-87.641697,"(41.671475169, -87.641696924)"
51,8316912,HT550919,10/19/2011 09:30:00 PM,096XX S GREEN ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENTIAL YARD (FRONT/BACK),False,False,...,21,73,14,1172345,1840843,2011,02/04/2016 06:33:39 AM,41.718713,-87.644359,"(41.718712866, -87.64435905)"
57,8316919,HT551141,10/20/2011 11:50:00 AM,111XX S ASHLAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,34,75,08B,1167337,1830440,2011,02/04/2016 06:33:39 AM,41.690274,-87.662998,"(41.690273866, -87.662998245)"
58,8316921,HT551082,10/20/2011 11:20:00 AM,100XX S MAY ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,True,...,34,73,14,1170428,1838136,2011,02/04/2016 06:33:39 AM,41.711326,-87.651459,"(41.711326335, -87.65145888)"


In [75]:
df.sort_values(["Latitude","Longitude"])

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
88,8316989,HT551163,10/20/2011 12:00:00 PM,132XX S CARONDOLET AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE-GARAGE,False,False,...,10,55,14,1198437,1817746,2011,02/04/2016 06:33:39 AM,41.654720,-87.549564,"(41.65471964, -87.549564253)"
44,8316899,HT550695,10/20/2011 05:30:00 AM,122XX S HALSTED ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE PORCH/HALLWAY,False,False,...,34,53,14,1173210,1823636,2011,02/04/2016 06:33:39 AM,41.671475,-87.641697,"(41.671475169, -87.641696924)"
57,8316919,HT551141,10/20/2011 11:50:00 AM,111XX S ASHLAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,34,75,08B,1167337,1830440,2011,02/04/2016 06:33:39 AM,41.690274,-87.662998,"(41.690273866, -87.662998245)"
61,8316927,HT550899,05/05/2011 09:00:00 AM,010XX W 109TH ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,34,75,11,1171191,1832528,2011,02/04/2016 06:33:39 AM,41.695921,-87.648828,"(41.695920504, -87.648827836)"
33,8316882,HT549162,10/19/2011 06:42:00 AM,105XX S WESTERN AVE,0610,BURGLARY,FORCIBLE ENTRY,TAVERN/LIQUOR STORE,False,False,...,19,72,05,1162255,1834721,2011,02/04/2016 06:33:39 AM,41.702129,-87.681485,"(41.702128701, -87.681485145)"
91,8316993,HT549180,10/18/2011 08:00:00 PM,106XX S HOXIE AVE,0810,THEFT,OVER $500,STREET,False,False,...,10,51,06,1195211,1834998,2011,02/04/2016 06:33:39 AM,41.702141,-87.560802,"(41.702140957, -87.560802048)"
34,8316884,HT544972,10/16/2011 04:30:00 AM,103XX S HALSTED ST,1310,CRIMINAL DAMAGE,TO PROPERTY,SMALL RETAIL STORE,False,False,...,34,73,14,1172807,1836192,2011,02/04/2016 06:33:39 AM,41.705940,-87.642804,"(41.705939683, -87.642803521)"
4,8316814,HT551005,10/20/2011 10:40:00 AM,103XX S ELIZABETH ST,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,21,73,08B,1169846,1836216,2011,02/04/2016 06:33:39 AM,41.706070,-87.653646,"(41.706070186, -87.653645803)"
12,8316839,HT551049,10/20/2011 08:50:00 AM,102XX S AVENUE N,0430,BATTERY,AGGRAVATED: OTHER DANG WEAPON,STREET,False,False,...,10,52,04B,1201166,1837763,2011,02/04/2016 06:33:39 AM,41.709580,-87.538904,"(41.709579698, -87.538903651)"
58,8316921,HT551082,10/20/2011 11:20:00 AM,100XX S MAY ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,True,...,34,73,14,1170428,1838136,2011,02/04/2016 06:33:39 AM,41.711326,-87.651459,"(41.711326335, -87.65145888)"


In [92]:
import cynet as cn